In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics

In [24]:
df = pd.read_csv('games_bought.csv')
df.head(5)

,Order_ID,Item_Name,Item_No,Price,Publisher,Rating,Sales Channel
0,1000,Grand Theft Auto,S18_1342,29.99,Activision Blizzard,3,Online
1,1000,Call of Duty: Vanguard,S20_1421,26.99,Activision Blizzard,5,Online
2,1000,Madden NFL23,N23_9809,35.99,EA,5,Online
3,1001,Animal Crossing,S21_4320,18.99,Nintendo,4,Retail
4,1001,Grand Theft Auto,S18_1342,29.99,Activision Blizzard,5,Retail


# 1. When a customer views a series, we need an affinity score (for all games) to recommend other items to the customer

In [25]:
# Building an affinity score is similar to finding correlation value
itemAffinity = pd.DataFrame(columns=('game1', 'game2', 'score'))
rowCount = 0
itemAffinity

,game1,game2,score


In [93]:
# We get our first list of unique items
# Set makes it unique
games=list(set(df['Item_Name']))

# Comparing each item in the list with other items
for i in range(len(games)):
    # Find out who bought game 1
    game1Users = df[df.Item_Name==games[i]]['Order_ID'].tolist()

    # If the item the loop is on encounters itself or an item already included
    # Skip over it

    for j in range(len(games)):
        if i == j:
            continue

        # If game is not the same, find out who bought game 2
        game2Users = df[df.Item_Name==games[j]]['Order_ID'].tolist()

        # Score will be the intersection of users who bought one game, who also bought another
        # Can imagine it like a Venn diagram
        purchasesInCommon=len(set(game1Users).intersection(set(game2Users)))

        # Score being how often this occurred throughout all the games
        score = purchasesInCommon / len(games)

        # This score: bought item 1? likelihood you bought item 2
        itemAffinity.loc[rowCount] = [games[i], games[j], score]
        rowCount +=1

        # This score: reverse of the above
        itemAffinity.loc[rowCount] = [games[j], games[i], score]
        rowCount +=1

# Find out what our final affinity score will be
itemAffinity.nlargest(10, 'score')




,game1,game2,score
4,Animal Crossing,Call of Duty: Vanguard,0.5
5,Call of Duty: Vanguard,Animal Crossing,0.5
8,Animal Crossing,FIFA 23,0.5
9,FIFA 23,Animal Crossing,0.5
12,Madden NFL23,Assassin's Creed,0.5
13,Assassin's Creed,Madden NFL23,0.5
16,Madden NFL23,Grand Theft Auto,0.5
17,Grand Theft Auto,Madden NFL23,0.5
22,Assassin's Creed,Madden NFL23,0.5
23,Madden NFL23,Assassin's Creed,0.5


# 2. Create an item list (for all games) that you can recommend to the user in order of preference (include the name of the Items, Prices, Publishers & Score).

In [196]:
# Want to have a list where we can index by a game name and get a recommendation list

# here's how we get one recommendation list:
searchItem = 'Animal Crossing'
recommendList = itemAffinity[itemAffinity.game1 == searchItem][["game2", "score"]].sort_values('score',ascending = [0]).round(2)

#print("Recommendations for Animal Crossing -Item #4")
recommendList.nlargest(5, "score")
display(recommendList.drop_duplicates())

recommendations = []
games_run = []
def buildRecommendedList(search_item):
        recommendation_list = (itemAffinity[itemAffinity.game1 == search_item][["game2", "score"]].sort_values('score',ascending = [0]).round(2)).drop_duplicates()
        games_run.append(search_item)
        return recommendation_list



for game in itemAffinity['game1']:
    if game in games_run:
        continue
    recommendations.append({game: pd.DataFrame(buildRecommendedList(game))})


def getList(search_term):
    for recommendation in recommendations:
        if search_term in str(recommendation.keys()):
            return list(recommendation.values())



gta = getList('Grand Theft Auto')
animal_crossing = getList('Animal Crossing')
animal_crossing

,game2,score
591,FIFA 23,0.50
151,Call of Duty: Vanguard,0.50
360,Madden NFL23,0.17
581,Grand Theft Auto,0.17
201,Assassin's Creed,0.00


[                      game2  score
 591                 FIFA 23   0.50
 151  Call of Duty: Vanguard   0.50
 360            Madden NFL23   0.17
 581        Grand Theft Auto   0.17
 201        Assassin's Creed   0.00]